In [4]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

import pandas as pd
import numpy as np 
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from pathlib import Path
import src.data_utils as u
from tqdm.auto import tqdm
from collections import Counter
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier

pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 100)

u.random_seed(42)

data_path = Path('../data')

In [ ]:
train_full, test, all_cols, cont_cols, cat_cols = u.read_data(data_path)

In [ ]:
for f in cont_cols:
    test[f].fillna(0, inplace=True)
    train_full[f].fillna(0, inplace=True)

In [ ]:
val_start = '2018-10-01'

train = u.select_by_time(train_full, '2016-01-01', val_start, 'datetime')
val = u.select_by_time(train_full, val_start, '2018-12-31', 'datetime')

In [ ]:
clf = RandomForestClassifier(n_estimators=150, max_depth=6, random_state=42, class_weight={0: 0.01, 1: 0.99})
clf.fit(train[cont_cols], train['y'])

In [ ]:
probas = clf.predict_proba(val[cont_cols])
u.f1_flexible(probas[:, 1], val['y'], 0, 0.99, 50)

In [ ]:
encoder = sklearn.preprocessing.OrdinalEncoder()

In [ ]:
encoder.fit_transform(X=val[cat_cols])

In [ ]:
encoder.transform(X=val[cat_cols])